In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
import tf_idf
import ELMo
import bag_of_words
import BERT_updated
import fasttext
import wget

ModuleNotFoundError: No module named 'wget'

In [35]:
main_data = pd.read_csv('/Users/andrewsimon/Desktop/Dow_dat.csv')

In [36]:
embeddings_tfidf = tf_idf.generate_tfidf_embeddings(main_data, 'Report', 'Level')

In [37]:
embeddings_array = embeddings_tfidf.drop(columns='Level').values
labels = embeddings_tfidf['Level'].values

In [38]:
X_train, X_test, y_train, y_test = train_test_split(embeddings_array,
                                                    labels, test_size=0.2, random_state=42)

In [39]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1100).fit(X_train, y_train)
predictions = rf.predict(X_test)
score = rf.score(X_test, y_test)
score

0.7

In [40]:
embeddings_BoW = bag_of_words.generate_bow_embeddings(main_data, 'Report', "Level")

In [42]:
embeddings_array = embeddings_BoW.drop(columns='Level').values
labels=embeddings_BoW["Level"].values

In [43]:
X_train, X_test, y_train, y_test = train_test_split(embeddings_array,
                                                    labels, test_size=0.2, random_state=42)

In [49]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1100).fit(X_train, y_train)
predictions = rf.predict(X_test)
score = rf.score(X_test, y_test)
score

0.8333333333333334

In [52]:
fasttext_embeddings = fasttext.fasttext_embedding(main_data, 'Report', 'Level')

In [64]:
fasttext_embeddings

,Embeddings,Labels
0,"[0.004625462461262941, -0.0037768115289509296,...",2
1,"[0.0057347905822098255, -0.015659162774682045,...",3
2,"[0.005751278251409531, -0.009470653720200062, ...",4
3,"[-0.0010858145542442799, -0.03006475232541561,...",1
4,"[0.013210098259150982, -0.019099511206150055, ...",5
...,...,...
295,"[0.010301179252564907, -0.02050667442381382, 0...",3
296,"[0.010560682974755764, -0.018368281424045563, ...",5
297,"[0.004984517581760883, -0.027650432661175728, ...",4
298,"[0.0015107534127309918, -0.01813441887497902, ...",2


In [68]:
X_train, X_test, y_train, y_test = train_test_split(fasttext_embeddings['Embeddings'].values,
                                                    fasttext_embeddings['Labels'], test_size=0.2, random_state=42)

In [69]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1100).fit(X_train, y_train)
predictions = rf.predict(X_test)
score = rf.score(X_test, y_test)
score

ValueError: setting an array element with a sequence.

In [18]:
bert_embeddings = BERT_updated.generate_bert_embeddings(main_data, 'Report', 'Level')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings['Embeddings'].values,
                                                    bert_embeddings['Labels'].values, test_size=0.2, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1100).fit(X_train, y_train)
predictions = rf.predict(X_test)
score = rf.score(X_test, y_test)
score

ValueError: setting an array element with a sequence.